In [1]:
import os
import requests
import json
from dotenv import load_dotenv
from openai import OpenAI

In [2]:
load_dotenv()
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
AIDEVS_API_KEY = os.environ["AIDEVS_API_KEY"]

In [3]:
client = OpenAI()

In [4]:
# First, check if the endpoint exists
response = requests.get("https://api.nbp.pl/api/exchangerates/tables/A/")
response_json = json.loads(response.text)
print(response_json[0]["rates"])

def get_exchange_rate(currency_name: str) -> float:
    response = requests.get("https://api.nbp.pl/api/exchangerates/tables/A/")
    response_json = json.loads(response.text)
    rates = response_json[0]["rates"]
    mids = [rate["mid"] for rate in rates if rate["currency"] == currency_name]
    
    if not mids:
        raise Exception(f"Couldn't find any exchange rates for currency: {currency_name}")
    return mids[0]

get_exchange_rate_def = {
    "type": "function",
    "function": {
        "name": "get_exchange_rate",
        "description": "Returns exchange rate for a given currency in relation to PLN",
        "parameters": {
            "type": "object",
            "properties": {
                "currency_name": {
                    "type": "string",
                    "description": "Currency name in Polish in lowercase.",
                },
            },
            "required": ["currency_name"],
        },
    },
}

[{'currency': 'bat (Tajlandia)', 'code': 'THB', 'mid': 0.109}, {'currency': 'dolar amerykański', 'code': 'USD', 'mid': 4.0245}, {'currency': 'dolar australijski', 'code': 'AUD', 'mid': 2.6353}, {'currency': 'dolar Hongkongu', 'code': 'HKD', 'mid': 0.5141}, {'currency': 'dolar kanadyjski', 'code': 'CAD', 'mid': 2.9492}, {'currency': 'dolar nowozelandzki', 'code': 'NZD', 'mid': 2.3989}, {'currency': 'dolar singapurski', 'code': 'SGD', 'mid': 2.9598}, {'currency': 'euro', 'code': 'EUR', 'mid': 4.3225}, {'currency': 'forint (Węgry)', 'code': 'HUF', 'mid': 0.011002}, {'currency': 'frank szwajcarski', 'code': 'CHF', 'mid': 4.4145}, {'currency': 'funt szterling', 'code': 'GBP', 'mid': 5.0414}, {'currency': 'hrywna (Ukraina)', 'code': 'UAH', 'mid': 0.1016}, {'currency': 'jen (Japonia)', 'code': 'JPY', 'mid': 0.025711}, {'currency': 'korona czeska', 'code': 'CZK', 'mid': 0.1719}, {'currency': 'korona duńska', 'code': 'DKK', 'mid': 0.5796}, {'currency': 'korona islandzka', 'code': 'ISK', 'mid': 

In [5]:
# First, check if the endpoint exists
response = requests.get("https://restcountries.com/v3.1/name/poland?fields=name,currencies,population")
response_json = json.loads(response.text)
print(response_json)

def get_country_population(country_name: str) -> int:
    response = requests.get(f"https://restcountries.com/v3.1/name/{country_name}?fields=name,currencies,population")
    response_json = json.loads(response.text)
    return response_json[0]["population"]

get_country_population_def = {
    "type": "function",
    "function": {
        "name": "get_country_population",
        "description": "Returns country's population",
        "parameters": {
            "type": "object",
            "properties": {
                "country_name": {
                    "type": "string",
                    "description": "Country name in English in lowercase.",
                },
            },
            "required": ["country_name"],
        },
    },
}

[{'name': {'common': 'Poland', 'official': 'Republic of Poland', 'nativeName': {'pol': {'official': 'Rzeczpospolita Polska', 'common': 'Polska'}}}, 'currencies': {'PLN': {'name': 'Polish złoty', 'symbol': 'zł'}}, 'population': 37950802}]


In [6]:
functions = {
    "get_exchange_rate": get_exchange_rate,
    "get_country_population": get_country_population
}

In [7]:
response = requests.post("https://tasks.aidevs.pl/token/knowledge", json=dict(apikey=AIDEVS_API_KEY))
response_json = json.loads(response.text)
token = response_json["token"]

response = requests.get(f"https://tasks.aidevs.pl/task/{token}")
response_json = json.loads(response.text)
print(response_json)
question = response_json["question"]

{'code': 0, 'msg': 'I will ask you a question about the exchange rate, the current population or general knowledge. Decide whether you will take your knowledge from external sources or from the knowledge of the model', 'question': 'ile orientacyjnie ludzi mieszka w Polsce?', 'database #1': 'Currency http://api.nbp.pl/en.html (use table A)', 'database #2': "Knowledge about countries https://restcountries.com/ - field 'population'"}


In [8]:
system_context_agent = """
You are an agent that decides whether the given question is about currency exchange rate or country's current population.
If the question doesn't match any of this categories you simply return "general". So the only allowed values you can return
are 'exchange', 'population' or 'general'.

### Example 1
User: ile orientacyjnie ludzi mieszka w Polsce?
Asistant: 'population'

### Example 2
User: jak nazywa się stolica Macedonii Pólnocnej?
Asistant: 'general'

### Example 3
User: aktualny kurs Franka Szwajcarskiego
Asistant: 'exchange'
"""

system_context_general = """
You are an agent specialized in answering general questions in most concise way possible. It means you return only
the answer to given question and nothing more.

### Example 2
User: jak nazywa się stolica Macedonii Pólnocnej?
Asistant: Skopje
"""

In [9]:
completion = client.chat.completions.create(
  model="gpt-4",
  messages=[
    {"role": "system", "content": system_context_agent},
    {"role": "user", "content": question}
  ]
)

question_type = str(completion.choices[0].message.content)
print(question_type)

'population'


In [10]:
if question_type == "'general'":
    completion = client.chat.completions.create(
      model="gpt-4",
      messages=[
        {"role": "system", "content": system_context_general},
        {"role": "user", "content": question}
      ]
    )

    answer = str(completion.choices[0].message.content)
    print(answer)
else:
    completion = client.chat.completions.create(
      model="gpt-4",
      messages=[
        {"role": "user", "content": question}
      ],
      tools=[get_exchange_rate_def, get_country_population_def],
      tool_choice="auto",
    )

    message = completion.choices[0].message
    if message.tool_calls:
        func_name = message.tool_calls[0].function.name
        func_args = json.loads(message.tool_calls[0].function.arguments)
        print(func_name)
        print(func_args, end="\n\n")
        
        answer = functions[func_name](**func_args)
        print(answer)
    else:
        print(f"No tool_calls in function calling response: {message}")

get_country_population
{'country_name': 'poland'}

37950802


In [11]:
response = requests.post(f"https://tasks.aidevs.pl/answer/{token}", json=dict(answer=answer))
print(response.text)

{
    "code": 0,
    "msg": "OK",
    "note": "CORRECT"
}
